In [618]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

In [619]:
data = pd.read_csv('1830151.csv')

In [620]:
df=data.copy()

In [621]:
def con_date_time(df):
    df['clear_date']=pd.to_datetime(df['clear_date'])
    df['posting_date']=pd.to_datetime(df['posting_date'])
    df['due_in_date']=pd.to_datetime(df['due_in_date'],format='%Y%m%d', errors='ignore')
    df['document_create_date']=pd.to_datetime(df['document_create_date'],format='%Y%m%d', errors='ignore')
    df['document_create_date.1']=pd.to_datetime(df['document_create_date.1'],format='%Y%m%d', errors='ignore')
    df['baseline_create_date']=pd.to_datetime(df['baseline_create_date'], format='%Y%m%d', errors='ignore')
    df['buisness_year']=df['buisness_year'].astype(int)
    return None

In [622]:
con_date_time(df)

In [623]:
df.drop(['document_create_date'],axis=1,inplace=True)
df=df.sort_values(by='document_create_date.1',ignore_index=True)
df=df.rename(columns={'document_create_date.1':'document_create_date'})

In [624]:
data_train=df.loc[data['clear_date'].notna()]
data_train=data_train.reset_index(drop=True)
data_null_cleardate=df.loc[data['clear_date'].isna()]
data_null_cleardate=data_null_cleardate.reset_index(drop=True)

In [625]:
df=data_train
df=(df.loc[df['document_create_date']<=df['posting_date']].loc[df['posting_date']<=df['due_in_date']].loc[df['baseline_create_date']<=df['due_in_date']].loc[df['baseline_create_date']<=df['clear_date']])

In [626]:
df['due_duration']=(df['clear_date']-df['due_in_date']).dt.days
df = df.loc[df['due_duration']<10]

In [627]:
df=df.drop(['clear_date'],axis=1)

Initial EDA

In [628]:
for i in df.columns:
    if df[i].isna().any():
        print(i)

area_business


In [629]:
df=df.drop(['area_business','isOpen'],axis=1)

In [630]:
df['conv_amt']=df['total_open_amount'].copy()

df.loc[df['invoice_currency']=='CAD','conv_amt']=df.loc[df['invoice_currency']=='CAD','conv_amt']*0.78

In [631]:
df.drop(['invoice_currency'],axis=1,inplace=True)

In [632]:
x_train, x_test_init = train_test_split(
    df,
    test_size=0.3,
    shuffle=False)
x_train.shape,  x_test_init.shape

((30286, 16), (12980, 16))

In [633]:
x_val, x_test = train_test_split(
    x_test_init,
    test_size=0.5,
    shuffle=False)

x_val.shape, x_test.shape

((6490, 16), (6490, 16))

In [634]:
x_test.columns

Index(['business_code', 'cust_number', 'name_customer', 'buisness_year',
       'doc_id', 'posting_date', 'document_create_date', 'due_in_date',
       'document type', 'posting_id', 'total_open_amount',
       'baseline_create_date', 'cust_payment_terms', 'invoice_id',
       'due_duration', 'conv_amt'],
      dtype='object')

In [635]:
df=x_train.copy()
mapper1=df.groupby('cust_number')['due_duration'].mean()
mapper2=df.groupby('name_customer')['due_duration'].mean()


In [636]:
# df

In [637]:
# mapper1

In [638]:
df=x_train.copy()

mapper1=df.groupby('cust_number')['due_duration'].mean()
mapper2=df.groupby('name_customer')['due_duration'].mean()

df['cust_delay_mean']=df['cust_number'].map(mapper1)
df['cust_name_delay_mean']=df['name_customer'].map(mapper2)
df['payment_time']=(df['due_in_date']-df['document_create_date']).dt.days

#df.columns#.invoice_currency.value_counts()

a=df
code_business_code = {k: i for i, k in enumerate(a.business_code.unique(), 1)}
code_cust_payment_terms = {k: i for i, k in enumerate(a.cust_payment_terms.unique(), 1)}
code_cust_number = {k: i for i, k in enumerate(a.cust_number.unique(), 1)}
code_name_customer = {k: i for i, k in enumerate(a.name_customer.unique(), 1)}

def encoding(df_obj):
    df_obj.loc[:, 'code_business_code'] = df_obj.loc[:, 'business_code'].map(code_business_code)
    df_obj.loc[:, 'code_cust_payment_terms'] = df_obj.loc[:, 'cust_payment_terms'].map(code_cust_payment_terms)
    df_obj.loc[:, 'code_cust_number'] = df_obj.loc[:, 'cust_number'].map(code_cust_number)
    df_obj.loc[:, 'code_name_customer'] = df_obj.loc[:, 'name_customer'].map(code_name_customer)
    return None

def dates_feature(df_dates):
    df_dates['posting_date_month'],df_dates['posting_date_day'],df_dates['posting_date_year'],df_dates['posting_date_quarter'],df_dates['posting_date_dayofweek'],df_dates['posting_date_week']=df_dates['posting_date'].dt.month,df_dates['posting_date'].dt.day,df_dates['posting_date'].dt.year,df_dates['posting_date'].dt.quarter,df_dates['posting_date'].dt.dayofweek,df_dates['posting_date'].dt.isocalendar().week.astype(int)
    df_dates['due_in_date_month'],df_dates['due_in_date_day'],df_dates['due_in_date_year'],df_dates['due_in_date_quarter'],df_dates['due_in_date_dayofweek'],df_dates['due_in_date_week']=df_dates['due_in_date'].dt.month,df_dates['due_in_date'].dt.day,df_dates['due_in_date'].dt.year,df_dates['due_in_date'].dt.quarter,df_dates['due_in_date'].dt.dayofweek,df_dates['due_in_date'].dt.isocalendar().week.astype(int)
    df_dates['document_create_date_month'],df_dates['document_create_date_day'],df_dates['document_create_date_year'],df_dates['document_create_date_quarter'],df_dates['document_create_date_dayofweek'],df_dates['document_create_date_week']=df_dates['document_create_date'].dt.month,df_dates['document_create_date'].dt.day,df_dates['document_create_date'].dt.year,df_dates['document_create_date'].dt.quarter,df_dates['document_create_date'].dt.dayofweek,df_dates['document_create_date'].dt.isocalendar().week.astype(int)
    df_dates['baseline_create_date_month'],df_dates['baseline_create_date_day'],df_dates['baseline_create_date_year'],df_dates['baseline_create_date_quarter'],df_dates['baseline_create_date_dayofweek'],df_dates['baseline_create_date_week']=df_dates['baseline_create_date'].dt.month,df_dates['baseline_create_date'].dt.day,df_dates['baseline_create_date'].dt.year,df_dates['baseline_create_date'].dt.quarter,df_dates['baseline_create_date'].dt.dayofweek,df_dates['baseline_create_date'].dt.isocalendar().week.astype(int)
    return None

encoding(df)
dates_feature(df)

#df['document_create_date'].dt.day_name()
df['document_create_is_working']=np.where(df.document_create_date_dayofweek.isin([5,6]),0,1)
df['due_in_is_working']=np.where(df.due_in_date_dayofweek.isin([5,6]),0,1)

In [639]:
# mapper1

In [640]:
drop=['posting_date','document_create_date','due_in_date','baseline_create_date','name_customer','cust_number','cust_payment_terms','business_code']

In [641]:
# drop

In [642]:
df=df.drop(drop,axis=1)

In [643]:
modes = {}
for i in df.columns:
    modes[i]=df[i].mode()

In [644]:
df.columns.value_counts().sum()

41

In [645]:
Xtr=df.copy()

In [646]:
q=x_val.copy()

In [647]:
# q.drop(['clear_date','invoice_currency'],axis=1,inplace=True)

In [648]:
# q=q.drop(['area_business','isOpen'],axis=1)

In [649]:
df.head(1)

,buisness_year,doc_id,document type,posting_id,total_open_amount,invoice_id,due_duration,conv_amt,cust_delay_mean,cust_name_delay_mean,...,document_create_date_dayofweek,document_create_date_week,baseline_create_date_month,baseline_create_date_day,baseline_create_date_year,baseline_create_date_quarter,baseline_create_date_dayofweek,baseline_create_date_week,document_create_is_working,due_in_is_working
1,2019,1.928537e+09,RV,1.0,48419.67,1.928537e+09,0,48419.67,0.383178,1.142857,...,6,52,12,30,2018,4,6,52,0,1


In [650]:
drop.append('document type')

In [651]:
drop.append('posting_id')

In [652]:
# mapper1

In [653]:
df=q
df['cust_delay_mean']=df['cust_number'].map(mapper1)
df['cust_name_delay_mean']=df['name_customer'].map(mapper2)
df['payment_time']=(df['due_in_date']-df['document_create_date']).dt.days
encoding(df)
dates_feature(df)
df['document_create_is_working']=np.where(df.document_create_date_dayofweek.isin([5,6]),0,1)
df['due_in_is_working']=np.where(df.due_in_date_dayofweek.isin([5,6]),0,1)
df=df.drop(drop,axis=1)
df.columns.value_counts().sum()

39

In [654]:
# df

In [655]:
for i in df.columns:
    if df[i].isna().any():
        df[i].fillna(int(modes[i]),inplace=True)

for i in df.columns:
    if df[i].isna().any():
        print(i)

In [656]:
Xval=df.copy()

In [657]:
Xtr_bkp=Xtr.copy()
Xval_bkp=Xval.copy()

In [658]:
Xtr=Xtr_bkp.copy()
Xval=Xval_bkp.copy()

Ytr= Xtr['due_duration'].copy()
Xtr.drop(['due_duration'],axis=1,inplace=True)

Yval= Xval['due_duration'].copy()
Xval.drop(['due_duration'],axis=1,inplace=True)

In [659]:
#list(zip(list(Xtr.columns),[1]*len(list(Xtr.columns))))

In [660]:
feature_selection=[      ('buisness_year', 1),
                         ('doc_id', 1),
                         ('document type', 0),
                         ('posting_id', 0),
                         ('total_open_amount', 1),
                         ('invoice_id', 1),
                         ('conv_amt', 1),
                         ('cust_delay_mean', 1),
                         ('cust_name_delay_mean', 1),
                         ('payment_time', 1),
                         ('code_business_code', 1),
                         ('code_cust_payment_terms', 1),
                         ('code_cust_number', 1),
                         ('code_name_customer', 1),
                         ('posting_date_month', 1),
                         ('posting_date_day', 1),
                         ('posting_date_year', 1),
                         ('posting_date_quarter', 1),
                         ('posting_date_dayofweek', 1),
                         ('posting_date_week', 1),
                         ('due_in_date_month', 1),
                         ('due_in_date_day', 1),
                         ('due_in_date_year', 1),
                         ('due_in_date_quarter', 1),
                         ('due_in_date_dayofweek', 1),
                         ('due_in_date_week', 1),
                         ('document_create_date_month', 1),
                         ('document_create_date_day', 1),
                         ('document_create_date_year', 1),
                         ('document_create_date_quarter', 1),
                         ('document_create_date_dayofweek', 1),
                         ('document_create_date_week', 1),
                         ('baseline_create_date_month', 1),
                         ('baseline_create_date_day', 1),
                         ('baseline_create_date_year', 1),
                         ('baseline_create_date_quarter', 1),
                         ('baseline_create_date_dayofweek', 1),
                         ('baseline_create_date_week', 1)]

In [661]:
feature=[]
for i in feature_selection:
    if i[1]==1:
        feature.append(i[0])

In [662]:
Xtr=Xtr[feature].copy()
Xval=Xval[feature].copy()

## Which model to select

In [663]:
# Defining Lists to Store in the Results and Names of Algorithms
MSE_Score = []
R2_Score = []
Algorithm = []

### **1. Linear Regression**

In [664]:
# Fitting Simple Linear Regression to the Training Set
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(Xtr, Ytr)

# Predicting the Test Set Results
predicted = clf.predict(Xval)

In [665]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))

### **2. Support Vector Regression**

In [666]:
# Fitting SVR to the Training Set
Algorithm.append('Support Vector Regression')
clf = SVR()
clf.fit(Xtr, Ytr)

# Predicting the Test Set Results
predicted = clf.predict(Xval)

KeyboardInterrupt: 

In [ ]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))

### **3. Decision Tree Regressor**

In [ ]:
# Fitting Decision Tree to the Training Set
Algorithm.append('Decision Tree Regressor')
clf = DecisionTreeRegressor()
clf.fit(Xtr, Ytr)

# Predicting the Test Set Results
predicted = clf.predict(Xval)

In [ ]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))

### **4. Random Forest Regressor**

In [ ]:
# Fitting Random Forest Regressor Tree to the Training Set
Algorithm.append('Random Forest Regressor')
clf = RandomForestRegressor()
clf.fit(Xtr, Ytr)

# Predicting the Test Set Results
predicted = clf.predict(Xval)

In [ ]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))

### **5. XGB Regressor**

In [ ]:
# Fitting XGBoost Regressor to the Training Set
Algorithm.append('XGB Regressor')
clf = xgb.XGBRegressor(objective ='reg:squarederror')
clf.fit(Xtr, Ytr)

# Predicting the Test Set Results
predicted = clf.predict(Xval)

In [ ]:
# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))

## Score Card of the Models' Performances

In [ ]:
# Just Combining the Lists into a DataFrame for a Better Visualisation
Comparison = pd.DataFrame(list(zip(Algorithm, MSE_Score, R2_Score)), columns = ['Algorithm', 'MSE_Score', 'R2_Score'])

In [ ]:
# Score Card
Comparison

## Random Forest

In [ ]:
MSE_Score = []
R2_Score = []
Algorithm = []
MAE_Score = []
RMSE_Score = []
isTrue = []
isFalse = []
AllFeatures = []

In [ ]:
Xtr=Xtr_bkp.copy()
Xval=Xval_bkp.copy()

In [570]:
# Xtr_bkp

In [571]:
Ytr= Xtr['due_duration'].copy()
Xtr.drop(['due_duration'],axis=1,inplace=True)

In [572]:
Yval= Xval['due_duration'].copy()
Xval.drop(['due_duration'],axis=1,inplace=True)

In [573]:
#list(zip(list(Xtr.columns),[1]*len(list(Xtr.columns))))

In [574]:
feature_selection=[      ('buisness_year', 0),
                         ('doc_id', 0),
                         ('document type', 0),
                         ('posting_id', 0),
                         ('total_open_amount', 0),
                         ('invoice_id', 0),
                         ('conv_amt', 1),
                         ('cust_delay_mean', 1),
                         ('cust_name_delay_mean', 0),
                         ('payment_time', 1),
                         ('code_business_code', 0),
                         ('code_cust_payment_terms', 1),
                         ('code_cust_number', 1),
                         ('code_name_customer', 0),
                         ('posting_date_month', 1),
                         ('posting_date_day', 1),
                         ('posting_date_year', 0),
                         ('posting_date_quarter', 0),
                         ('posting_date_dayofweek', 0),
                         ('posting_date_week', 0),
                         ('due_in_date_month', 1),
                         ('due_in_date_day', 1),
                         ('due_in_date_year', 0),
                         ('due_in_date_quarter', 1),
                         ('due_in_date_dayofweek', 1),
                         ('due_in_date_week', 0),
                         ('document_create_date_month', 1),
                         ('document_create_date_day', 1),
                         ('document_create_date_year', 0),
                         ('document_create_date_quarter', 1),
                         ('document_create_date_dayofweek', 1),
                         ('document_create_date_week', 0),
                         ('baseline_create_date_month', 1),
                         ('baseline_create_date_day', 1),
                         ('baseline_create_date_year', 0),
                         ('baseline_create_date_quarter', 0),
                         ('baseline_create_date_dayofweek', 0),
                         ('baseline_create_date_week', 0),
                         ('document_create_is_working', 1),
                         ('due_in_is_working', 1)]

In [575]:
feature=[]
for i in feature_selection:
    if i[1]==1:
        feature.append(i[0])

In [576]:
AllFeatures.append(feature)

In [577]:
Xtr=Xtr[feature].copy()
Xval=Xval[feature].copy()

In [578]:
# Xtr

In [579]:
def stan(a,n):
    if n==1:
        scaler = StandardScaler()
    elif n==2:
        scaler = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
    return scaler

In [580]:
df=Xtr
scaler=stan(df,2)
scaler.fit(df)
Xtr=scaler.transform(df)
df=Xval
scaler=stan(df,2)
scaler.fit(df)
Xval=scaler.transform(df)

In [581]:
# n_estimators=35,max_depth=30,min_samples_leaf=10,min_samples_split=5
# n_estimators=45,max_depth=35,min_samples_leaf=15,min_samples_split=10

In [582]:
valS=[]
TrS=[]
n_e=[]
m_d=[]
m_s=[]
m_l=[]

In [583]:
n_e.append(45)
m_d.append(55)
m_s.append(25)
m_l.append(15)

In [584]:
# Ytr

In [585]:
clf = RandomForestRegressor(n_estimators=n_e[-1],max_depth=m_d[-1],min_samples_leaf=m_s[-1],min_samples_split=m_l[-1])
qwerty=clf.fit(Xtr, Ytr)

In [586]:
# Xval=xv.copy()
# Xval=x_val.copy()
#Xval=xv.copy()

In [587]:
# Yval

In [588]:
print(qwerty.score(Xval, Yval))
valS.append(qwerty.score(Xval, Yval))
TrS.append(qwerty.score(Xtr, Ytr))
qwerty.score(Xtr, Ytr)

-0.17581967249990704


0.6177003173018261

In [589]:
pd.DataFrame(list(zip(valS,TrS,n_e,m_d,m_s,m_l)))

,0,1,2,3,4,5
0,-0.17582,0.6177,45,55,25,15


In [590]:
# import pickle
# with open("Xtr", 'rb') as f:
#     XXX = pickle.load(f)

# with open("Ytr", 'rb') as f:
#     YYY = pickle.load(f)

# XXXDATA=pd.DataFrame(XXX)

# YYYDATA=pd.DataFrame(YYY)

# XTRDATA=pd.DataFrame(Xval)

# YTRDATA=pd.DataFrame(YYY)

# XXXDATA.shape

# XTRDATA.shape

# XXXDATA

In [591]:
# XTRDATA

In [592]:
# for i in (XXXDATA==XTRDATA).columns:
#     print((XXXDATA==XTRDATA)[i].unique())

In [593]:
# for i in (YYYDATA==YTRDATA).columns:
#     print((YYYDATA==YTRDATA)[i].unique())

In [594]:
# predicted = clf.predict(Xtr)
# # predicted = clf.predict(Xval)
# MSE_Score.append(mean_squared_error(Ytr, predicted))
# R2_Score.append(r2_score(Ytr, predicted))
# MAE_Score.append(mean_absolute_error(Ytr, predicted))
# RMSE_Score.append(np.sqrt(mean_squared_error(Ytr, predicted)))
# A=((abs((pd.Series(np.around(predicted)).astype(int)-Ytr.reset_index(drop=True))))<4).value_counts().to_frame().T
# for i in A.columns:
#     if i==True:
#         isTrue.append(int(A[i]))
#     else:
#         isFalse.append(int(A[i]))


In [600]:
predicted1 = clf.predict(Xtr)
predicted = clf.predict(Xval)
MSE_Score.append(mean_squared_error(Yval, predicted))
R2_Score.append(r2_score(Yval, predicted))
MAE_Score.append(mean_absolute_error(Yval, predicted))
RMSE_Score.append(np.sqrt(mean_squared_error(Yval, predicted)))
A=((abs((pd.Series(np.around(predicted)).astype(int)-Yval.reset_index(drop=True))))<4).value_counts().to_frame().T
for i in A.columns:
    if i==True:
        isTrue.append(int(A[i]))
    else:
        isFalse.append(int(A[i]))
# Just Combining the Lists into a DataFrame for a Better Visualisation


In [601]:
Comparison = pd.DataFrame(list(zip(MSE_Score,RMSE_Score, R2_Score,MAE_Score,isTrue,isFalse,AllFeatures)), columns = ['MSE_Score','RMSE_Score', 'R2_Score','MAE_Score','isTrue','isFalse','Allfeatures'])

In [606]:
pd.DataFrame(predicted)

,0
0,-6.608108
1,8.780970
2,1.043811
3,-2.091756
4,0.153298
...,...
6868,0.229143
6869,10.817734
6870,-2.393924
6871,-2.706871


In [607]:
pd.DataFrame(Yval)

,due_duration
32187,-6
32188,3
32189,5
32190,2
32191,0
...,...
39056,-1
39057,4
39058,2
39059,-4


In [602]:
Xval.shape

(6873, 19)

In [603]:
isTrue

[5103, 5103]

In [604]:
# Score Card
Comparison[['MSE_Score','RMSE_Score', 'R2_Score','MAE_Score','isTrue','isFalse']]

,MSE_Score,RMSE_Score,R2_Score,MAE_Score,isTrue,isFalse
0,58.739291,7.664156,-0.17582,3.617768,5103,1770


In [486]:
# len(AllFeatures)

In [525]:
q=x_test.copy()

In [526]:
q.columns

Index(['business_code', 'cust_number', 'name_customer', 'buisness_year',
       'doc_id', 'posting_date', 'document_create_date', 'due_in_date',
       'document type', 'posting_id', 'total_open_amount',
       'baseline_create_date', 'cust_payment_terms', 'invoice_id',
       'due_duration', 'conv_amt'],
      dtype='object')

In [527]:
# x_test

In [528]:
df=q
df['cust_delay_mean']=df['cust_number'].map(mapper1)
df['cust_name_delay_mean']=df['name_customer'].map(mapper2)
df['payment_time']=(df['due_in_date']-df['document_create_date']).dt.days
encoding(df)
dates_feature(df)
df['document_create_is_working']=np.where(df.document_create_date_dayofweek.isin([5,6]),0,1)
df['due_in_is_working']=np.where(df.due_in_date_dayofweek.isin([5,6]),0,1)
df=df.drop(drop,axis=1)
df.columns.value_counts().sum()

39

In [529]:
for i in df.columns:
    if df[i].isna().any():
        df[i].fillna(int(modes[i]),inplace=True)

for i in df.columns:
    if df[i].isna().any():
        print(i)

In [530]:
df.columns

Index(['buisness_year', 'doc_id', 'total_open_amount', 'invoice_id',
       'due_duration', 'conv_amt', 'cust_delay_mean', 'cust_name_delay_mean',
       'payment_time', 'code_business_code', 'code_cust_payment_terms',
       'code_cust_number', 'code_name_customer', 'posting_date_month',
       'posting_date_day', 'posting_date_year', 'posting_date_quarter',
       'posting_date_dayofweek', 'posting_date_week', 'due_in_date_month',
       'due_in_date_day', 'due_in_date_year', 'due_in_date_quarter',
       'due_in_date_dayofweek', 'due_in_date_week',
       'document_create_date_month', 'document_create_date_day',
       'document_create_date_year', 'document_create_date_quarter',
       'document_create_date_dayofweek', 'document_create_date_week',
       'baseline_create_date_month', 'baseline_create_date_day',
       'baseline_create_date_year', 'baseline_create_date_quarter',
       'baseline_create_date_dayofweek', 'baseline_create_date_week',
       'document_create_is_working',

In [536]:
Xte=df.copy()

Yte= Xte['due_duration'].copy()

Xte.drop(['due_duration'],axis=1,inplace=True)
Xte=Xte[feature].copy()

In [537]:
# Yval= Xte['due_duration'].copy()
# Yval

In [540]:
Yte

39061    2
39062   -1
39063    3
39064   -4
39065    0
        ..
45952    5
45953    0
45954    2
45955    0
45956    4
Name: due_duration, Length: 6873, dtype: int64

In [611]:
# predicted1 = clf.predict(Xtr)
predicted = clf.predict(Xte)
print(mean_squared_error(Yte, predicted))
print(r2_score(Yte, predicted))
print(mean_absolute_error(Yte, predicted))
print(np.sqrt(mean_squared_error(Yte, predicted)))
A=((abs((pd.Series(np.around(predicted)).astype(int)-Yte.reset_index(drop=True))))<4).value_counts().to_frame().T
for i in A.columns:
    if i==True:
        print('True',int(A[i]))
    else:
        print('False',int(A[i]))

1424.3333722719503
-13.708151527610234
29.113214345846366
37.74034144349982
False 5655
True 1218


In [612]:
aaa=pd.DataFrame(predicted,columns=["aaa"])

In [617]:
Xte

,conv_amt,cust_delay_mean,payment_time,code_cust_payment_terms,code_cust_number,posting_date_month,posting_date_day,due_in_date_month,due_in_date_day,due_in_date_quarter,due_in_date_dayofweek,document_create_date_month,document_create_date_day,document_create_date_quarter,document_create_date_dayofweek,baseline_create_date_month,baseline_create_date_day,document_create_is_working,due_in_is_working
39061,13617.0400,1.549521,15,12.0,48.0,12,9,12,24,4,1,12,9,4,0,12,9,1,1
39062,137722.9500,-1.000000,15,1.0,160.0,12,9,12,24,4,1,12,9,4,0,12,9,1,1
39063,44164.8200,1.385455,15,1.0,12.0,12,9,12,24,4,1,12,9,4,0,12,9,1,1
39064,2237.4300,-2.138366,15,3.0,4.0,12,9,12,24,4,1,12,9,4,0,12,9,1,1
39065,26435.1100,0.635207,15,6.0,35.0,12,9,12,24,4,1,12,9,4,0,12,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45952,34693.7214,9.449848,11,2.0,2.0,2,26,3,8,1,6,2,26,1,2,2,27,1,0
45953,4304.3700,0.406250,15,1.0,38.0,2,26,3,12,1,3,2,26,1,2,2,26,1,1
45954,73652.2300,1.000000,7,50.0,768.0,2,26,3,4,1,2,2,26,1,2,2,26,1,1
45955,50210.2600,0.697674,15,1.0,50.0,2,26,3,12,1,3,2,26,1,2,2,26,1,1


In [613]:
aaa.loc[aaa["aaa"]>10]

,aaa
9,36.967424
16,52.425210
19,64.223919
26,73.089621
27,73.089621
...,...
6854,38.459486
6856,73.089621
6861,53.214247
6868,73.089621


In [614]:
pd.DataFrame(Yte)

,due_duration
39061,2
39062,-1
39063,3
39064,-4
39065,0
...,...
45952,5
45953,0
45954,2
45955,0


In [543]:
# Just Combining the Lists into a DataFrame for a Better Visualisation
Comp = pd.DataFrame(list(zip(MSE_Score,RMSE_Score, R2_Score,MAE_Score,isTrue,isFalse,AllFeatures)), columns = ['MSE_Score','RMSE_Score', 'R2_Score','MAE_Score','isTrue','isFalse','Allfeatures'])
# Score Card
Comp[['MSE_Score','RMSE_Score', 'R2_Score','MAE_Score','isTrue','isFalse']]

,MSE_Score,RMSE_Score,R2_Score,MAE_Score,isTrue,isFalse
0,50.039831,7.073884,-0.001677,3.50773,5068,1805


In [513]:
# len(AllFeatures)

1

In [515]:
# Xte